# Homework 4 - pandas (Seth Romero)  
### due 11/17/2022   

Use this notebook and prompts to complete the homework. Throughout there will be hints and some code provided  

### Things you will need:  
- Install os, NumPy, pandas  
- states_covid.csv  
- Bloom_etal_2018_Reduced_Dataset.csv  
- logfiles.tgz (or some other multiple file dataset)  

*NOTE*: Make sure your PATH is correct  

**import packages & check required datasets**   

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
PATH = '/mnt/c/Users/Seth/Documents/GitHub/F22_BIOL792_coursepage/week11_python6/'

In [3]:
assert os.path.exists(os.path.join(PATH,'states_covid.csv')), 'states_covid.csv does not exist' 

In [4]:
assert os.path.exists(os.path.join(PATH,'Bloom_etal_2018_Reduced_Dataset.csv')), 'Bloom_etal_2018_Reduced_Dataset.csv does not exist'

## Task 1 - DataFrame manipulation  

Using **states_covid.csv**, we are going to read the data in as a DataFrame to manipulate, subset, and filter in various ways.   

### Task 1.1 

Read in states_covid.csv with date as a "date" dtype, and only columns consisting of the hospitalization (4 col), ICU (2 col), and Ventilators (2 col)  

In [5]:
col_opts = ['hosp', 'icu', 'vent', 'date', 'state']
sub_col = lambda x: any(name in x.casefold() for name in col_opts)
covid_states_df = pd.read_csv('states_covid.csv',
                             usecols = sub_col,
                             parse_dates = ['date'])
covid_states_df.dtypes

date                      datetime64[ns]
state                             object
hospitalized                     float64
hospitalizedCumulative           float64
hospitalizedCurrently            float64
hospitalizedIncrease               int64
inIcuCumulative                  float64
inIcuCurrently                   float64
onVentilatorCumulative           float64
onVentilatorCurrently            float64
dtype: object

In [6]:
covid_states_df.head()

,date,state,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,inIcuCurrently,onVentilatorCumulative,onVentilatorCurrently
0,2021-02-23,AK,1260.0,1260.0,38.0,9,NaN,NaN,NaN,5.0
1,2021-02-23,AL,45250.0,45250.0,762.0,122,2632.0,NaN,1497.0,NaN
2,2021-02-23,AR,14617.0,14617.0,545.0,47,NaN,204.0,1505.0,99.0
3,2021-02-23,AS,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
4,2021-02-23,AZ,57072.0,57072.0,1515.0,78,NaN,447.0,NaN,266.0


### Task 1.2 

For each of the following catergories: *currently* hospitalized, *currently* in the ICU, and *currently* on ventilation...  
Find the 5 states with the greatest numbers in each catergory and list them in decreasing order.      
*hint*: sort_values, unique  

In [7]:
def top5(df, col, group):
    return df.sort_values(by=col, ascending=False).drop_duplicates(subset=group)[['date', group, col]].head(5)
    
top5(covid_states_df, 'hospitalizedCurrently', 'state')

,date,state,hospitalizedCurrently
2637,2021-01-07,CA,22851.0
17733,2020-04-13,NY,18825.0
2399,2021-01-12,TX,14218.0
12162,2020-07-21,FL,9520.0
17618,2020-04-15,NJ,8270.0


In [8]:
top5(covid_states_df, 'inIcuCurrently', 'state')

,date,state,inIcuCurrently
17677,2020-04-14,NY,5225.0
2413,2021-01-11,CA,4971.0
1951,2021-01-20,TX,3686.0
17674,2020-04-14,NJ,2051.0
17888,2020-04-10,MI,1663.0


In [9]:
top5(covid_states_df, 'onVentilatorCurrently', 'state')

,date,state,onVentilatorCurrently
16389,2020-05-07,NY,2425.0
17618,2020-04-15,NJ,1705.0
17832,2020-04-11,MI,1441.0
3958,2020-12-15,OH,863.0
17768,2020-04-12,IL,821.0


### Task 1.3 

For each state, identify the date when the state reached 1000 cumulative hospitilized covid patients

In [10]:
# some states don't have a record of cumulative hospitalization, date of threshold may be before date of record or just unknown...
hosp1000_w_value = covid_states_df[covid_states_df.hospitalizedCumulative >= 1000]
w_value = hosp1000_w_value.sort_values(by='date', ascending=True).drop_duplicates(subset='state').sort_values(by='state', ascending=True)

# creating a df with also possible states (including Guam, Virgin Islands, Puerto Rico, American Samoa, and Northern Mariana Islands)
# to be joined with states with values
all_states = pd.DataFrame(covid_states_df['state'].unique(), columns = ['state'])

# merging and formatting
full = pd.merge(all_states, w_value, on='state', how='outer')
full.sort_values(by='date')[['state', 'date']]

,state,date
37,NY,2020-03-21
10,FL,2020-04-02
11,GA,2020-04-02
38,OH,2020-04-04
41,PA,2020-04-04
21,MA,2020-04-04
22,MD,2020-04-06
6,CO,2020-04-07
4,AZ,2020-04-11
53,WI,2020-04-14


## Task 2 - DataFrame summarizing  

Using **Bloom_etal_2018_Reduced_Dataset.csv**, we are going to do more dataframe manipulation and subsetting and summarizing    

### Task 2.1 

Read in Bloom_etal_2018_Reduced_Dataset.csv and create two new columns ('genus','species') that consists of the column *taxa* split at the underscore. Print out the head of this new dataframe and the number of unique genera**     

*hint:* pd.str.split(,expand=True)  

for example:  

| taxa | genus | species   
| :------ | :-- | :---   
| Alosa_alabamae | Alosa | alabamae  

In [11]:
bloom_species = pd.read_csv("Bloom_etal_2018_Reduced_Dataset.csv")
bloom_species[["genus", "species", "subspecies"]] = bloom_species.taxa.str.split("_", expand=True)
print(bloom_species.head(), "\n\nUnique genera =", bloom_species.genus.unique().size)

                   taxa  logbodysize  trophic_position         Reg  genus  \
0        Alosa_alabamae     1.707570          0.431364  diadromous  Alosa   
1           Alosa_alosa     1.778151          0.556303  diadromous  Alosa   
2          Alosa_fallax     1.778151          0.556303  diadromous  Alosa   
3       Alosa_mediocris     1.778151          0.612784  diadromous  Alosa   
4  Alosa_pseudoharengus     1.602060          0.544068  diadromous  Alosa   

          species subspecies  
0        alabamae       None  
1           alosa       None  
2          fallax       None  
3       mediocris       None  
4  pseudoharengus       None   

Unique genera = 34


### Task 2.2 

Create a new dataframe with the mean *logbodysize* and *trophicposition* of each genera. Sort this data frame by the largest body size. Print the head of this dataframe.

In [12]:
bloom_sizes = bloom_species.groupby('genus').agg(avg_size=('logbodysize', 'mean'), avg_trophic=('trophic_position', 'mean')).sort_values(by='avg_size', ascending=False)
bloom_sizes.head()

,avg_size,avg_trophic
genus,,
Tenualosa,1.778151,0.462398
Alosa,1.739062,0.540815
Coilia,1.544068,0.477121
Ethmalosa,1.544068,0.397940
Potamalosa,1.505150,0.518514


Which genera is the smallest and largest?

In [13]:
print("Largest genera: %s\nSmallest genera: %s" %(bloom_sizes.index[0], bloom_sizes.index[-1]))

Largest genera: Tenualosa
Smallest genera: Amazonsprattus


What is the trophic position of the smallest and largest?  

In [14]:
print("Largest genera: %s (Trophic pos. = %.2f)\nSmallest genera: %s (Trophic pos. = %.2f)" %(bloom_sizes.index[0], bloom_sizes.iat[0,1], bloom_sizes.index[-1], bloom_sizes.iat[-1,1]))

Largest genera: Tenualosa (Trophic pos. = 0.46)
Smallest genera: Amazonsprattus (Trophic pos. = 0.53)


## Task 3 - Read in muliple files to a dictionary and make a DataFrame - **OPTIONAL/BONUS**  

### This is not something you are expected to do in this course, but just here to give you an idea of the things that you COULD do. Answers will be posted after due date.  


Using **logfiles**: we are going to do read in each file, get some data, append it to a dictionary to later make into a dataframe.     

**note:** *make sure to unzip logfiles*

In [50]:
cd $PATH

/mnt/c/Users/Seth/Documents/GitHub/F22_BIOL792_coursepage/week11_python6


In [53]:
!tar -xf logfiles.tgz #unzip logfiles

In [54]:
log_dir = os.path.join(PATH,'logfiles')
assert os.path.exists(log_dir), 'log_dir does not exist'

First step is to find the necessary files. The number of files in the log files is 36, make sure you have that many as well  

In [55]:
!ls -l logfiles/*txt | wc -l 

36


In [65]:
logfiles = !find $log_dir -name '*txt' #unix command to find files in log_dir directory
logfiles = [os.path.abspath(x) for x in logfiles] #this finds the full path to the file
print(logfiles[36])

/mnt/c/Users/Seth/Documents/GitHub/F22_BIOL792_coursepage/week11_python6/logfiles/1901302108_H7_D_14.txt.txt


In [57]:
assert(len(logfiles)==36), 'Do not have correct number of logfiles'

AssertionError: Do not have correct number of logfiles

### Getting a little tricky here  

Read in each of the logfiles, for each file extract:  
- minimum temperature  
- maximum temperature  
- date of minimum temp   
- date of maximum temp   
- mean temp for each file.   

This data should all be appended for a dictionary within a for loop:    
Key should be the file name without the path or .txt extension  
Values should be (minTemp,maxTemp,minDate,maxDate,meanTemp)

I recommend making this work for one file first, then putting the rest in a for loop to do the rest.  

Below is an example of how to read in one file

*hint:* do not read date in as date object

In [85]:
### set up data frame as you read in each file
infile = pd.read_csv(logfiles[36],sep='\t',engine='python',encoding='windows-1252',
                    parse_dates = ['Date', 'Time'])
infile.columns = ['Index','Date','Time','Temp','Type']
infile = infile[['Date','Time','Temp']]
infile['Date'] = 
infile.head(40)

,Date,Time,Temp
0,2013-09-29,2022-11-15 08:00:00,32.4
1,2013-09-29,2022-11-15 08:35:00,32.5
2,2013-09-29,2022-11-15 09:10:00,32.5
3,2013-09-29,2022-11-15 09:45:00,32.6
4,2013-09-29,2022-11-15 10:20:00,32.8
5,2013-09-29,2022-11-15 10:55:00,32.9
6,2013-09-29,2022-11-15 11:30:00,33.0
7,2013-09-29,2022-11-15 12:05:00,33.0
8,2013-09-29,2022-11-15 12:40:00,33.2
9,2013-09-29,2022-11-15 13:15:00,33.3


Do everything in steps, make sure it works. Calculate summaries with this one infile:

In [70]:
# minTemp =   
# maxTemp =  
minDate = infile['Date'][infile['Temp'] == infile['Temp'].min()].unique()[0] #use this for minDate 
maxDate = infile['Date'][infile['Temp'] == infile['Temp'].max()].unique()[0] #use this for maxDate 
# meanTemp = 
minDate

'11/5/2013'

To get you started, I suggest writing some dummy code in plain words to help outline your for loop:  

In [ ]:
logfiles_dict = {}  
for f in logfiles:  
    #do something   
    #do not read date in as date object  
    #do more something   
    #do other stuff  
    #make print statements EVERYWHERE  
    #append to dict  
    #blahbahblah

Then do the real code below here. You don't need to turnin your thoughts. Just put it in there as a help reminder. Most people all still do this, no matter how advanced they are

**Once you have created a DataFrame with all the logfiles, print the head and save it to an outfile using pd.to_csv() as logfiles_df.csv** 

#### Here is an example of the final product

NameError: name 'logfiles_df' is not defined